# Google Analytics Kaggle Challenge

## Import the libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn import linear_model
import itertools as it
import re

In [3]:
import sys
import csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

## Define variables

In [4]:
data_path = 'D:/000_Projects_2018/0002_Development/Kaggle/google_analytics/data/'

## Define functions

In [5]:
def separate_json(series: pd.Series) -> pd.DataFrame():
    """
    
    Args:
        series: Series before json parsing 

    Returns: DataFrame

    """
    # TODO: Write TypeException
    
    if isinstance(series[0], str):
        return pd.DataFrame(json.loads(s) for s in series)
    return pd.DataFrame(s for s in series)

## Extract Transform Load: ETL

In [89]:
#json_col = ['customDimensions','device', 'geoNetwork', 'totals', 'trafficSource']
json_col = ['customDimensions','device', 'geoNetwork', 'totals', 'trafficSource',
            'trafficSource.adwordsClickInfo','customDimensions.0',
           'trafficSource.adwordsClickInfo.targetingCriteria']
nest_json_col = ['hits']
df_train = pd.read_csv('{}train_v2.csv'.format(data_path) ,nrows=2000,engine='python')
df_train = df_train.drop('hits',axis=1)
df_train ['customDimensions']= df_train['customDimensions'].map(lambda x: str(x).replace("\'", "\""))
for col_name in json_col:
    print(col_name)
    if col_name == 'customDimensions.0':
        df_test[col_name]=df_test[col_name].\
        map(lambda x: {'index':'None'} if x is None else x)

    df = separate_json(df_train[col_name])
    df.columns = ['{}.{}'.format(col_name,x) for x in list(df.columns)]

    df_train = df_train.join(df)
    df_train = df_train.drop(col_name,axis=1)
#df_train.columns

df_test = pd.read_csv('{}test_v2.csv'.format(data_path),nrows=2000,engine='python')
df_test ['customDimensions']= df_test['customDimensions'].map(lambda x: str(x).replace("\'", "\""))
df_test = df_test.drop('hits',axis=1)

for col_name in json_col:
    print(col_name)
    if col_name == 'customDimensions.0':
        df_test[col_name]=df_test[col_name].\
        map(lambda x: {'index':'None'} if x is None else x)

    df = separate_json(df_test[col_name])
    df.columns = ['{}.{}'.format(col_name,x) for x in list(df.columns)]

    df_test = df_test.join(df)
    df_test = df_test.drop(col_name,axis=1)

customDimensions
device
geoNetwork
totals
trafficSource
trafficSource.adwordsClickInfo
customDimensions.0
trafficSource.adwordsClickInfo.targetingCriteria
customDimensions
device
geoNetwork
totals
trafficSource
trafficSource.adwordsClickInfo
customDimensions.0
trafficSource.adwordsClickInfo.targetingCriteria


In [32]:
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace('"', "'"))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("True", '"True"'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("False", '"False"'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("{'", '{"'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("'}", '"}'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("':", '":'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace(": '", ': "'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace(", '", ', "'))
#df_train ['hits']= df_train['hits'].map(lambda x: str(x).replace("',", '",'))

In [14]:
#print(len(df_train.loc[1,'hits']))
#print(df_train.loc[443,'hits'][7450  :7550  ])


In [15]:
#df = separate_json(df_train[nest_json_col[0]])

In [16]:
#df.head(2)

In [17]:
#print(np.sum(df.isnull(),axis=1))

In [18]:
#x = df[df[0].isnull()==False].index
#df_t = df.loc[x,0]
#df_t.index = range(len(df_t))
#print(x)
#df1 = separate_json(df_t)

In [19]:
#df1.head()

In [20]:
#df2 = separate_json(df1['contentGroup'])

In [76]:
df_train['customDimensions.0']=df_train['customDimensions.0'].map(lambda x: '{"index":"1"}' if '{' not in str(x) else x)


In [97]:
x = df_train[df_train['trafficSource.adwordsClickInfo.targetingCriteria.0'].isnull()==False].index
#df_train['trafficSource.adwordsClickInfo.targetingCriteria.0'].astype('object').value_counts()
print(df_train.loc[x,'trafficSource.adwordsClickInfo.targetingCriteria.0'])

337    {}
464    {}
Name: trafficSource.adwordsClickInfo.targetingCriteria.0, dtype: object


In [114]:
print("The train dataset has {} columns".format(len(df_train.columns)))
print("The test dataset has {} columns".format(len(df_test.columns)))
print("The number of rows in the train dataset:{}".format(len(df_train)))
print("The number of rows in the test dataset:{}".format(len(df_test)))

The train dataset has 39 columns
The test dataset has 39 columns
The number of rows in the train dataset:2000
The number of rows in the test dataset:2000


In [23]:
#df_train_naidu = pd.read_csv('{}extracted_fields_train.csv'.format(data_path), engine='python')
#df_test_naidu = pd.read_csv('{}extracted_fields_test.csv'.format(data_path), engine='python')

In [24]:
#print("The train dataset has {} columns".format(len(df_train_naidu.columns)))
#print("The test dataset has {} columns".format(len(df_test_naidu.columns)))
#print("The number of rows in the train dataset:{}".format(len(df_train_naidu)))
#print("The number of rows in the test dataset:{}".format(len(df_test_naidu)))

In [108]:
for column in list(df_train.columns):
    if column not in list(df_test.columns):
        print(column)

In [115]:
def f4(x):
    print("x: column nr.{} in the training set which is {}".format(x,list(df_train.columns)[x]))
    print(df_train[list(df_train.columns)[x]].value_counts(dropna=False))

interact(f4, x=widgets.IntSlider(min=0,max=len(df_train.columns)-1,step=1,value=0));


interactive(children=(IntSlider(value=0, description='x', max=38), Output()), _dom_classes=('widget-interact',))

In [116]:
x =df_train[df_train['totals.transactionRevenue'].fillna(0).astype('int64') > 0].index
#df_train.loc[x,'source'].value_counts(dropna=False)

In [ ]:
import re
X=df_train['source'].map(lambda x: (re.match(r'.*youtube.*', str(x).lower()) is not None)*1)

In [ ]:
Y=X*df_train['transactionRevenue'].fillna(0).astype('int64')
Y.head()

In [ ]:
sum(Y>0)/sum(df_train['transactionRevenue'].fillna(0).astype('int64')>0)

In [ ]:
print(re.match(r'.*go+gle.*', str('+Google6').lower()))

In [120]:
def f5(x):
    print("x: column nr.{} in the test set which is {}".format(x,list(df_test.columns)[x]))
    print(df_test[list(df_test.columns)[x]].value_counts(dropna=False))

interact(f5, x=widgets.IntSlider(min=0,max=len(df_test.columns)-1,step=1,value=0));

interactive(children=(IntSlider(value=0, description='x', max=38), Output()), _dom_classes=('widget-interact',))

## Data aggregation

In [ ]:
print(len(df_train.groupby(['fullVisitorId']).sum()))
print(len(df_test.groupby(['fullVisitorId']).sum()))

In [ ]:
fullVisitorId_test = df_test.fullVisitorId

In [113]:
#try:
const_cols_train = [c for c in df_train.columns
                  if df_train[c].nunique(dropna=False)==1 ]
const_cols_test = [c for c in df_test.columns 
                 if df_test[c].nunique(dropna=False)==1 ]

# Drop the columns with constant values
df_train = df_train.drop(const_cols_train,
                                     axis=1, inplace=False)
df_test = df_test.drop(const_cols_test,
                                   axis=1, inplace=False)
#except:
#print("problem with the columns")

In [119]:
len(df_test.columns)

39

In [ ]:
expected_column_to_detet=['fullVisitorId','socialEngagementType','browserSize','browserVersion','flashVersion',
                         'language','mobileDeviceBranding','mobileDeviceInfo','mobileDeviceMarketingName',
                         'mobileDeviceModel','mobileInputSelector','operatingSystemVersion','screenColors',
                          'screenResolution','cityId','latitude','longitude','networkLocation','visits',
                          'campaignCode','criteriaParameters','targetingCriteria']

In [121]:
def one_hot(df,column):
    
    df_ohc = pd.get_dummies(df[column])
    df_ohc.columns = ['_'.join((column, str(x))) for x in range(len(df_ohc.columns))]
    df = pd.concat([df,df_ohc],axis = 1)
   
    return df

In [122]:
columns = ['channelGrouping','device.deviceCategory',
           'geoNetwork.continent', 'geoNetwork.subContinent','trafficSource.medium']

for column in columns:
    
    df_train = one_hot(df_train,column)
    df_test = one_hot(df_test,column)
    print(column)
    print(len(df_train.columns))
    print(len(df_test.columns))

channelGrouping
46
46
device.deviceCategory
49
49
geoNetwork.continent
55
55
geoNetwork.subContinent
74
74
trafficSource.medium
80
80


In [ ]:
print("The clean train dataset has {} columns".format(len(df_train.columns)))
print("The clean test dataset has {} columns".format(len(df_test.columns)))

In [ ]:
train_revenue = df_train[~df_train['transactionRevenue'].isnull()].copy()
print(len(train_revenue))

In [ ]:
try:
    Y_train = df_train.transactionRevenue
    df_train = df_train.drop(['transactionRevenue'],axis = 1)
except:
    print('the target already deleted')

## Replace the strings with integers

In [ ]:
def replace_strings_integer(df_train, df_test):
    df_total = pd.concat([df_train,df_test],sort=False)
    df_total.index=range(len(df_total['date']))
    df_train_decoded = df_train
    df_test_decoded= df_test
    for col_i in df_train.columns[df_train.dtypes == 'object']:
            
        df_total[col_i] = df_total[col_i].factorize()[0]
        df_train_decoded[col_i] = df_total.loc[range(df_train.shape[0]),col_i].values
        df_test_decoded[col_i] =  df_total.loc[range(df_train.shape[0],
                                                     df_train.shape[0]+df_test.shape[0]),
                                               col_i].values
    return df_train_decoded, df_test_decoded

In [ ]:
df_train, df_test = replace_strings_integer(df_train, df_test)

In [ ]:
Y_train = Y_train.fillna(0)
print(Y_train.value_counts(dropna=False))

In [ ]:
train_revenue = Y_train[Y_train.astype('int64')>0].copy()
min(train_revenue)
np.log1p(min(train_revenue.astype('int64')))

In [ ]:
## interactive tool
col_train = df_train.columns[df_train.dtypes != 'object']
col_test = df_test.columns[df_test.dtypes != 'object']
print(len(col_train))
print(len(col_test))

def f(x,print_feature=False):
    
    compare_columns(x,df_train,df_test,print_feature)

In [ ]:
df_corr_train = df_train.fillna(0).corr()
df_corr_test = df_test.fillna(0).corr()

In [ ]:
def compare_columns(col_i,df_train,df_test,print_feature):
    
    col_train = df_train.columns[df_train.dtypes != 'object']
    col_test = df_test.columns[df_test.dtypes != 'object']

    print('the column {} in both training and testing sets are: '.format(col_i) + 
          col_train[col_i] +' , ' + col_test[col_i])
    print('the mean value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].mean()) +' , ' + str(df_test[col_test[col_i]].mean()))
    print('the std value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].std()) +' , ' + str(df_test[col_test[col_i]].std()))
    print('the max value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].max()) +' , ' + str(df_test[col_test[col_i]].max()))
    print('the min value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].min()) +' , ' + str(df_test[col_test[col_i]].min()))
    print('Number of uniqe values of the column {}: '.format(col_i) + 
          str(len(df_train[col_train[col_i]].value_counts())) + '/' + str(len(df_train[col_train[col_i]])) + ' , ' +
          str(len(df_test[col_test[col_i]].value_counts())) + '/' + str(len(df_test[col_test[col_i]])))
    print('Number of uniqe values of the column {} in both datasets compared to the uniqe values in train set: '.format(col_i) + 
          str(len(pd.concat([df_train[col_train[col_i]],df_test[col_test[col_i]]]).value_counts())) + '/' +
         str(len(df_train[col_train[col_i]].value_counts())))
    
    print('The most frequent values of the column {} in both datasets: \n'.format(col_i) + 
          str(df_train[col_train[col_i]].value_counts().head()) + ' , \n' +
         str(df_test[col_test[col_i]].value_counts().head()))
    print ('correlation between the feature and the target')
    print(np.corrcoef([df_train[col_train[col_i]].fillna(0),Y_train.astype('int64')]))
    plt.matshow(df_corr_train)
    
    plt.matshow(df_corr_test)
    if print_feature:
        
        plt.scatter(df_train[col_train[0]],df_train[col_train[col_i]],c=Y_train)
    
    print(df_corr_train.loc[df_corr_train[col_train[col_i]]>0.85,col_train[col_i]].head())
    print(df_corr_test.loc[df_corr_test[col_train[col_i]]>0.85,col_train[col_i]].head())

In [ ]:
interact(f, x=widgets.IntSlider(min=0,max=32,step=1,value=0));

## insights:
hists,pageviews have 15 % correlation with the target

In [ ]:
#extra_columns_to_delete = ['visitStartTime']   

In [ ]:
#df_train, df_test = drop_unnecessary_columns(extra_columns_to_delete,df_train,df_test)

In [ ]:
#print("The clean train dataset has {} columns".format(len(df_train.columns)))
#print("The clean test dataset has {} columns".format(len(df_test.columns)))

In [ ]:
Y_train_b = (Y_train.astype('int64') > 0)*1

In [ ]:
print(Y_train_b.value_counts(dropna=False))

In [ ]:
#random_state = 0
#x_train, x_cv, y_train, y_cv= train_test_split(df_train,Y_train_b,
#                       test_size=0.1,stratify=Y_train_b,
#                       random_state=random_state)

In [ ]:
#print(y_train.value_counts(dropna=False))

In [ ]:
#print(y_cv.value_counts(dropna=False))

In [ ]:
#clf = RandomForestClassifier(n_estimators=200,max_depth=15,random_state=0,n_jobs=-1)
#clf.fit(x_train, y_train)
#print('train:',clf.score(x_train, y_train))

In [ ]:
#print('cross-validation:',clf.score(x_cv, y_cv))

## Apply classification on the test data:

In [ ]:
#Y_test_b = clf.predict(df_test)

In [ ]:
#print(sum(Y_test_b))
#print(len(Y_test_b))

In [ ]:
#Y_train_n = Y_train[Y_train.astype('int64')>0].astype('int64')
#df_train_n = df_train[Y_train.astype('int64')>0]

In [ ]:
#print(Y_train_n.value_counts(dropna=False))

In [ ]:
#def f_1(x):
#    print(df_train_n.columns[x])
#    print('original')
#    print(np.corrcoef([df_train_n[df_train_n.columns[x]],Y_train_n]))
#    print('log')
#    print(np.corrcoef([df_train_n[df_train_n.columns[x]]**2,np.log(Y_train_n)]))
    
#interact(f_1, x=widgets.IntSlider(min=0,max=31,step=1,value=0));

In [ ]:
#column_to_delete = ['bounces']

In [ ]:
#df_train_n, _ = drop_unnecessary_columns(column_to_delete,df_train_n,df_test)

## Normaize the data

In [ ]:
#df_train_n_n=(df_train_n-np.mean(df_train_n))/np.std(df_train_n)
#df_test_n

In [ ]:
#linreg = linear_model.LinearRegression()
#linreg.fit(df_train_n_n, Y_train_n)

In [ ]:
#print(linreg.score(df_train_n_n,Y_train_n))

In [ ]:
#plt.plot(df_train_n_n[df_train_n_n.columns[12]],np.log(Y_train_n),'.')

In [ ]:
def map_features(X, map_degree,maped_fea):
    V=np.zeros((len(maped_fea),1))
    cor_f=pd.DataFrame(maped_fea)
    com_x_f=[]
    for i in range(2,map_degree+1):
        com_x=list(it.combinations_with_replacement(range(1,10), i))#(range(n_x), i))
        for j in range(len(com_x)):
            if com_x[j][0]!=com_x[j][1] or com_x[j][0]!=com_x[j][1]:
                V[:,0]= X[:,com_x[j][0]]*X[:,com_x[j][1]]
                cor_f['V']=V
                X=np.append(X.T,np.array(X[:,com_x[j][0]]*X[:,com_x[j][1]]).reshape(1,-1),axis=0).T
                com_x_f.append(com_x[j])
    return X,com_x_f

In [ ]:
#x_train_map,com_x_f=map_features(np.array(df_train_n_n),2,Y_train_n)

In [ ]:
#linreg.fit(x_train_map, Y_train_n)

In [ ]:
#print(linreg.score(x_train_map,Y_train_n))

In [ ]:
#x_train_m, x_cv_m, y_train_m, y_cv_m= train_test_split(x_train_map,Y_train_n,
#                       test_size=0.1,random_state=random_state)

In [ ]:
import lightgbm as lgb 

lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 200, "learning_rate" : 0.01, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
#lgb_train = lgb.Dataset(x_train_m, label=np.log(y_train_m))
#lgb_val = lgb.Dataset(x_cv_m, label=np.log(y_cv_m))
#model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=150, verbose_eval=20)

In [ ]:
random_state = 0
x_train, x_cv, y_train, y_cv= train_test_split(df_train,Y_train.astype('int64'),
                       test_size=0.1,stratify=Y_train_b,random_state=random_state)

In [ ]:
x_train_map,com_x_f=map_features(np.array(x_train),2,y_train)

In [ ]:
def map_features_test(X, com_x_f):
    com_x=com_x_f
    for j in range(len(com_x)):
        X=np.append(X.T,np.array(X[:,com_x[j][0]]*X[:,com_x[j][1]]).reshape(1,-1),axis=0).T
    return X

In [ ]:
x_cv_map=map_features_test(np.array(x_cv), com_x_f)

In [ ]:
lgb_train = lgb.Dataset(x_train_map, label=np.log1p(y_train))
lgb_val = lgb.Dataset(x_cv_map, label=np.log1p(y_cv))
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=150, verbose_eval=20)

In [ ]:
x_test_map=map_features_test(np.array(df_test), com_x_f)

In [ ]:
x_test_map.shape

In [ ]:
df_test.shape

In [ ]:
preds = model.predict(x_test_map, num_iteration=model.best_iteration)

In [ ]:
print(len(preds))

In [ ]:
pred_sub = pd.DataFrame(fullVisitorId_test)

In [ ]:
#pred_sub = pd.DataFrame(columns=['fullVisitorId','PredictedLogRevenue'])
#pred_sub.fullVisitorId = fullVisitorId_test
pred_sub['PredictedLogRevenue'] = preds

In [ ]:
x =pred_sub[pred_sub.PredictedLogRevenue < 5].index

In [ ]:
pred_sub.loc[x,'PredictedLogRevenue']=0
pred_sub.head()

In [ ]:
#pred_sub.to_csv(data_path+'predict.csv',index=False)

In [ ]:
#pred_sub_agg = pred_sub.groupby(['fullVisitorId']).sum()

In [ ]:
#pred_sub_agg.head()

In [ ]:
#print(len(pred_sub_agg))

In [ ]:
#pred_sub_agg.to_csv('{}initial_submission_5.csv'.format(data_path))

In [ ]:
df_external = pd.read_csv('{}SYB61_T29_Internet Usage.csv'.format(data_path), engine='python')
df_external.head(20)

In [ ]:
def f4(x):
    print("x: column nr.{} in the external set which is {}".format(x,list(df_external.columns)[x]))
    print(df_external[list(df_external.columns)[x]].value_counts(dropna=False))

interact(f4, x=widgets.IntSlider(min=0,max=len(df_external.columns)-1,step=1,value=0));